<center>![](https://cdn-images-1.medium.com/max/600/1*2kpd1gySE1aKCL6MS4YvPg.jpeg)

In [5]:
# %load_ext autoreload
# %autoreload 2

# !apt-get update 
# !apt-get install -y cmake swig python-opengl ffmpeg xvfb

# !pip install joblib six
# !pip install 'gym[all]'
# !pip install pyglet==1.2.4
# !pip install pyvirtualdisplay
# !pip install --no-cache-dir -I pillow

In [6]:
# # создаем фиктивный дисплей:
# !pkill -9 Xvfb
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()

In [8]:
import matplotlib.pyplot as plt
import gym
import numpy as np
import scipy
import matplotlib.animation as animation 
# from google.colab import files

def make_video(frames):
    # Cоздаем окно, в которое будем писать кадры:
    fig = plt.figure()
    plt.axis('off')
    im = plt.imshow([[0]])

    # Конвертируем кадры в видео:
    video = animation.FuncAnimation(fig, im.set_data, frames)
    video.save('demo.mp4', writer=animation.FFMpegWriter(), dpi=100)

    # Cкачиваем файлы с colab:
    files.download('demo.mp4')

In [9]:
# Будем играть в MountainCar-v0 https://github.com/openai/gym/wiki/MountainCar-v0
env = gym.make('MountainCar-v0').env
env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


array([-0.56391219,  0.        ])

In [10]:
# Создадим агента

class Agent:
    def __init__(self, params=None):
        if params is None:
            self.params = np.random.normal(
                0, 1 / (env.observation_space.high - env.observation_space.low)) * 10
        else:
            self.params = np.array(params)
    def get_action(self, observarion):
        proba = scipy.special.expit(np.sum(observarion * self.params))
        if proba < 0.7:
            return 0
        if proba > 0.3:
            return 2
        return 1
    def mutate(self, mutation_rate):
        if np.random.normal() < mutation_rate:
            shift = np.random.normal(
                0, 1 / (env.observation_space.high - env.observation_space.low)) / 10
            self.params += shift
    
def crossing_over(agent_left, agent_right):
    new_params = []
    for param_index in range(env.observation_space.shape[0]):
        new_params.append(
            np.random.choice((agent_left.params[param_index], 
                              agent_right.params[param_index])))
    return Agent(new_params)

In [11]:
# Начнем эволюцию!

agents_count = 1000
agents_pool = [Agent() for _ in range(agents_count)] # создаем колонию из 1000 агентов (особей)
# epoch_mean_rewards = []


for epoch_i in range(2):                  # сколько эпох (поколений) пройдет
    agents_rewards = []
    for agent_i, agent in enumerate(agents_pool):
        agent_rewards = []
        for i_episode in range(5):         # играем несколько эпизодов
            observation = env.reset()      # сбрасываем env – это нужно делать перед каждой игрой
            agent_reward = 0 
            for t in range(200):           # делаем макс. 200 шагов внутри эпизода
                action = agent.get_action(observation)
                observation, reward, done, info = env.step(action)   # совершаем шаг
                agent_reward += reward
                if done:                   # если игра закончилась – перестаем совершать шаги
                    break
            agent_rewards.append(agent_reward)
        agents_rewards.append(np.mean(agent_rewards))
    print('Epoch:', epoch_i, '| Mean reward:', np.mean(agents_rewards))
    plt.hist(agents_rewards)
    plt.show()
    best_agent_idexes = np.array(agents_rewards).argsort()[-30:][::-1]  # отбираем лучших агентов
    
    new_agents_pool = []
    
    for _ in range(agents_count):
        first_agent = agents_pool[np.random.choice(best_agent_idexes)]
        second_agent = agents_pool[np.random.choice(best_agent_idexes)]
        new_agent = crossing_over(first_agent, second_agent)            # скрещиваем 
        new_agent.mutate(mutation_rate=0.5)
        new_agents_pool.append(new_agent)
        
    agents_pool = new_agents_pool

AttributeError: module 'scipy' has no attribute 'special'

In [ ]:
# Выбираем лучшую особь

best_agent_i = np.array(agents_rewards).argmax()
agents_rewards[best_agent_i]

In [ ]:
def download_last_video():
    import os
    import time
    from google.colab import files

    dirname = 'videos'
    files_sorted = [x[0] for x in sorted([(fn, os.stat(dirname + '/' + fn)) 
                                   for fn in os.listdir(dirname)], 
                                  key = lambda x: -x[1].st_ctime)]
    video_found = False
    for file in files_sorted:
        if file.endswith(".mp4"):
            files.download(dirname + '/' + file)
            video_found = True
            break
    if not video_found:
        print('Video not found:', files_sorted)

In [ ]:
# Играем несколько игр и записываем

import gym.wrappers
env = gym.wrappers.Monitor(gym.make('MountainCar-v0'),directory="videos",force=True)

for i_episode in range(30):
    observation = env.reset()
    for t in range(200):
        action = agents_pool[best_agent_i].get_action(observation)
        observation, reward, done, info = env.step(action)
        agent_reward += reward
        if done:                   # если игра закончилась – перестаем совершать шаги
            break
    
env.close()
download_last_video()

In [ ]:
frames = []
# for i_episode in range(10):
observation = env.reset()
agent_reward = 0 
for t in range(200):
    frames.append(env.render(mode='rgb_array'))
    action = agents_pool[4].get_action(observation)
    observation, reward, done, info = env.step(action)
    agent_reward += reward 
    if done:                   # если игра закончилась – перестаем совершать шаги
        print('Done')
        break
        
print(agent_reward)
            
make_video(frames)